# Python 3.7 - News & tricks

------------

## What's new in Python 3.6?


### PEP 498: Formatted string literals (f-strings)

Nuevo tipo de string, **se prefijan con f**. Ejemplo:

In [2]:
f'Hola!' == 'Hola!'

True

Se pueden incluir variables entre llaves:

In [2]:
day, month, year = 4, 6, 2019

print('Using old format: {}/{}/{}'.format(day, month, year))

print(f'Using new style: {day}/{month}/{year}')

Using old format: 4/6/2019
Using new style: 4/6/2019


Lo que va entre `{}` es una expresión que se evalúa en tiempo de ejecución, así que podemos hacer cosas más complejas, como por ejemplo llamadas a funciones.

In [7]:
from googletrans import Translator

msg = '量力是就识知'  # Crypted chinese message

decrypt = lambda m: m[::-1]  # Just reverse a text

print(f'Decrypted message: {Translator().translate(decrypt(msg)).text!r}')  # `!r` calls the __repr__() method

Decrypted message: 'knowledge is power'


También mantiene el lenguaje que usa `format()`

>Más info: https://docs.python.org/3/library/string.html#formatspec

In [61]:
times = [1.0457234, 1.4353234, 3.20439, 2.341]
format_val = lambda val: f'{val:^{10}.{4}}'

# Header
header = ' '.join(f'{elem + str(i):^{10}}' for i, elem in enumerate(['ex'] * 4))
print(header)
print('=' * len(header))

# {var:{width}.{precision}}     also < align left, ^ align center
print(' '.join(format_val(t) for t in times))
print(' '.join(format_val(t*2) for t in times))

   ex0        ex1        ex2        ex3    
  1.046      1.435      3.204      2.341   
  2.091      2.871      6.409      4.682   


¿Cuándo usar `format()`, cuándo `+` y `%`? No hay una regla, pero Google nos tira una idea en su guía de estilo, basandose en la legibilidad del código.

>Ver: http://google.github.io/styleguide/pyguide.html#310-strings

### PEP 526: Syntax for variable annotations

¿Se acuerdan de las anotaciones? Volvieron... en forma de fichas!

![fichas](https://thumbs.gfycat.com/UnselfishQuarrelsomeGarpike-size_restricted.gif)

Ahora se pueden anotar variables sin ningún efecto en las mismas.

In [98]:
one: int = 1
name: str

print(__annotations__)

class Alf:
    forma: str = 'fichas'

print(Alf.__annotations__)

{'one': <class 'int'>, 'name': <class 'str'>}
{'forma': <class 'str'>}


Para forzar el tipado estático, ver `mypy` y `pytype`.

### PEP 515: Underscores in Numeric Literals

Se agrega la opción de usar `_` para leer mejor los números.

In [79]:
m = 1_000_000

f'{m} {m:_}'

'1000000 1_000_000'

### PEP 525: Asynchronous Generators



In [10]:
import asyncio

async def ticker(delay, to):
    """Yield numbers from 0 to *to* every *delay* seconds."""
    for i in range(to):
        yield i
        await asyncio.sleep(delay)

### PEP 530: Asynchronous Comprehensions


In [12]:
result = [i async for i in ticker(0.1, 10) if i % 2]
result

[1, 3, 5, 7, 9]

### PEP 487: Simpler customization of class creation

Nuevo método para inicializar subclases sin necesidad de usar una metaclase.

In [8]:
class PluginBase:
    subclasses = []

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        cls.subclasses.append(cls)
        print(cls.subclasses)

class Plugin1(PluginBase):
    pass

class Plugin2(PluginBase):
    pass

[<class '__main__.Plugin1'>]
[<class '__main__.Plugin1'>, <class '__main__.Plugin2'>]


### PEP 520: Preserving Class Attribute Definition Order

Se puede asumir que los atributos de una clase están ordenados como aparecen en el código, en el atributo `__dict__`.

In [25]:
class A:
    a = 0

    def b(self):
        pass

    c = 0

print(A.__dict__)

{'__module__': '__main__', 'a': 0, 'b': <function A.b at 0x7fd545c2dc80>, 'c': 0, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}


### PEP 468: Preserving Keyword Argument Order

`**kwargs` están ordenados de acuerdo a la inserción.


In [22]:
def a(**kwargs):
    print(kwargs)

a(a=0, b=0, c=0, d=0)

{'a': 0, 'b': 0, 'c': 0, 'd': 0}


### PYTHONMALLOC environment variable



### The Order of Dictionaries Is Guaranteed (insertion order)

In [25]:
a = {1: 0, 3: 0, 100: 0, -23: 0}
print(a)

{1: 0, 3: 0, 100: 0, -23: 0}


>Más info: https://docs.python.org/3/whatsnew/3.6.html


------------------------------

## What's new in Python 3.7?



### PEP 563: Postponed Evaluation of Annotations

Ahora se postpone la evaluación de anotaciones, lo que permite referencias a definiciones posteriores, por ejemplo:

In [18]:
from __future__ import annotations

class C:
    @classmethod
    def from_string(cls, source: str) -> C:
        pass

    def validate_b(self, obj: B) -> bool:
        pass

class B:
    pass


Esto rompe compatibilidad, por ende hay que activarlo con el import. Pasará a ser default en Python 4.0.

### PEP 553: Built-in `breakpoint()`

Hay una nueva función built-in para debugear. El debugger que va a usar por defecto es `pdb`, pero esto es configurable desde la variable de entorno `PYTHONBREAKPOINT`.

Ejemplo: `PYTHONBREAKPOINT=0` deshabilita esta funcionalidad, `PYTHONBREAKPOINT=ipdb.set_trace()` reemplaza por `ipdb`.

In [20]:
breakpoint()

--Call--
> /usr/local/lib/python3.7/site-packages/IPython/core/displayhook.py(252)__call__()
-> def __call__(self, result=None):
(Pdb) c


### Dataclasses

Nuevo módulo `dataclasses`.

Describe sus atributos con anotaciones de clase. Su constructor tanto como `__repr__()`, `__eq__()`, y `__hash__()` se generan automáticamente.

In [33]:
from dataclasses import dataclass, field

@dataclass(order=True)
class Country:
    name: str
    population: int
    area: float = field(repr=False, compare=False)
    coastline: float = 0

    def beach_per_person(self):
        """Meters of coastline per person"""
        return (self.coastline * 1_000) / self.population

norway = Country("Norway", 5_320_045, 323_802, 58_133)
usa = Country("United States", 326_625_791, 9_833_517, 19_924)

print(norway)
print(usa)

print(norway.beach_per_person())
print(usa.beach_per_person())

Country(name='Norway', population=5320045, coastline=58133)
Country(name='United States', population=326625791, coastline=19924)
10.927163210085629
0.06099946957342386


>Más info: https://docs.python.org/3/whatsnew/3.7.html

------------------

## Python tricks

### Merging two dicts in Python 3.5+

Una manera copada de mergear diccionarios en python.

In [29]:
x = {'a': 0, 'b': 1}
y = {'c': 2, 'd': 3}

z = {**x, **y}

print(z)

{'a': 0, 'b': 1, 'c': 2, 'd': 3}


### Usage of `any()` and `all()`

In [34]:
x, y, z = 0, 1, 0

# if x or y or z
if any((x, y, z)):
    print('Alguno es true')

# if x and y and z
if all((x, y, z)):
    print('Todos son true')

Alguno es true


### How to sort a Python dict by value

In [40]:
# How to sort a Python dict by value
# (== get a representation sorted by value)

xs = {'a': 4, 'b': 3, 'c': 2, 'd': 1}

sorted_xs = sorted(xs.items(), key=lambda x: x[1])
print(sorted_xs)

# Or:
import operator

sorted_xs = sorted(xs.items(), key=operator.itemgetter(1))
print(sorted_xs)

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]
[('d', 1), ('c', 2), ('b', 3), ('a', 4)]


### Python's namedtuples

In [42]:
from collections import namedtuple
Car = namedtuple('Car', 'color mileage')

# Our new "Car" class works as expected:
my_car = Car('red', 3812.4)
print(my_car.color)
print(my_car.mileage)

# We get a nice string repr for free:
print(my_car)

# Like tuples, namedtuples are immutable:
my_car.color = 'blue'

red
3812.4
Car(color='red', mileage=3812.4)


AttributeError: can't set attribute

### Swapping two variables

In [44]:
a = 111
b = 555

a, b = b, a

### Data pretty printer

In [8]:
from pprint import pprint

bin_data = {
    '402918': {
        'brand': 'VISA',
        'category': 'CLASSIC',
        'geo': 'AR',
        'organization': 'BANCO DE GALICIA Y BUENOS AIRES, S.A.',
        'type': 'DEBIT',
        'url_logos': ['static/bin/cards/banco-galicia.png', 'static/bin/cards/banco-galicia1.png']
    },
    '527608': {
        'brand': 'MASTERCARD',
        'category': 'BLACK',
        'geo': 'AR',
        'organization': 'BANCO MACRO, S.A.',
        'type': 'DEBIT',
        'url_logos': ['static/bin/cards/banco-macro.png']
    },
    '504649': {
        'brand': 'MAESTRO',
        'category': 'PERSONAL',
        'geo': 'GR',
        'organization': 'EUROBANK ERGASIAS, S.A.',
        'type': 'DEBIT',
    },
    '123456': {}
}

#print(bin_data)
pprint(bin_data, depth=1)

{'123456': {}, '402918': {...}, '504649': {...}, '527608': {...}}


### Dicts can be used to emulate switch/case statements

In [84]:
def dispatch_dict(operator, x, y):
    return {
        'add': lambda: x + y,
        'sub': lambda: x - y,
        'mul': lambda: x * y,
        'div': lambda: x / y,
    }.get(operator, lambda: None)()

print(dispatch_dict('add', 4, 3))
print(dispatch_dict('-', 4, 3))

7
None


### `+=` is faster

In [3]:
from timeit import timeit

t1 = timeit("s1 = s1 + s2 + s3", setup="s1 = ' ' * 100000; s2 = ' ' * 100000; s3 = ' ' * 100000", number=100)
t2 = timeit("s1 += s2 + s3", setup="s1 = ' ' * 100000; s2 = ' ' * 100000; s3 = ' ' * 100000", number=100)
print(t1)
print(t2)

0.18427745100052562
0.009104434997425415


### Chaining Comparison Operators

In [34]:
x = 5

print(1 < x < 10)
print(10 < x < 20)

print(x < 10 < x*10 < 100)
print(10 > x <= 9)
print(5 == x > 4)

True
False
True
True
True


### `enumerate()`

In [56]:
days = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sab', 'Dom']

for index, day in enumerate(days):#, start=1):
    print(index, day)

0 Lun
1 Mar
2 Mié
3 Jue
4 Vie
5 Sab
6 Dom


## Refs

Python 3.6

* https://docs.python.org/3/whatsnew/3.6.html
* https://www.python.org/dev/peps/pep-0498/
* http://google.github.io/styleguide/pyguide.html#310-strings

Python 3.7

* https://docs.python.org/3/whatsnew/3.7.html
* https://realpython.com/python37-new-features/

Python tricks

* https://realpython.com/
* https://github.com/satwikkansal/wtfpython
* https://stackoverflow.com/questions/101268/hidden-features-of-python